## Notebook used for constructing the datasets used in the empirical section

### 1) historical Standard & Poors 500 constituents dataset fetched from eodhd.com

In [175]:
import pickle
import os 

with open("eod_key.pkl", "rb") as input_file:
  eod_key = pickle.load(input_file)

from urllib.request import urlopen
import pandas as pd
import numpy as np
import urllib.request
from urllib.error import HTTPError

# Function to get the Data

def get_jsonparsed_data(url):
    res = urlopen(url)
    data = res.read().decode("utf-8")
    return json.loads(data)

def find_when_active(cons,tick):
    '''
    oututs a list of tuples containing the datetimes of when it first started and ended being part of the s&p500
    '''
    mask = cons['tickers'].apply(lambda x: tick in x)
    part_of = []
    active = False
    for idx in range(len(cons)):
        # first time appearing 
        if not active and mask.iloc[idx]:
            start    = mask.index[idx]
            active = True
            continue
        # removed from index
        if active and not mask.iloc[idx]:
            end      = mask.index[idx]
            active   = False
            part_of.append((start,end))
            continue

    if active:
        end = mask.index[-1]
        part_of.append((start,end))
    assert len(part_of) != 0
    return part_of

import datetime as dt
from tqdm import tqdm 

filename = 'price_df.csv'

# first look up the historical s&p 500 constituents
constituents = pd.read_csv('sp500.csv')
constituents.date  = pd.to_datetime(constituents.date)
constituents.set_index(constituents.date, inplace=True)
constituents.drop(columns=['date'],inplace=True)

constituents['tickers'] = constituents['tickers'].apply(lambda row: row.split(','))
all_tickers = set(np.concatenate(constituents.tickers.values))

begin = constituents.index[0]
end   = constituents.index[-1]

price_df = pd.DataFrame(index=pd.date_range(start=begin, end=end, freq="B"), columns = list(all_tickers))
failed_to_add = list()

# check if the file already exists
# if os.path.exists(filename):
#     price_df = pd.read_csv(filename)
#     price_df = price_df.rename(columns={'Unnamed: 0':'date'})
#     price_df.set_index('date', inplace=True)    

for count, ticker in enumerate(tqdm(all_tickers)):
    # find the correct dates the stock was part of the s&p 500
    for start, stop in find_when_active(constituents, ticker):
        # fetch data using eod api
        try:
            df  = get_jsonparsed_data("https://eodhd.com/api/eod/{}.US?from={}&to={}&period=d&api_token={}&fmt=json".format(ticker.replace('.','-'),
                start.strftime('%Y-%m-%d'),stop.strftime('%Y-%m-%d'), eod_key))
        except HTTPError as e:
            if e.code == 404:
                print('404 not found from api for ticker: {}'.format(ticker))
                failed_to_add.append(ticker)
                continue
            else:
                print(e)
        # output if no data was able to be found
        if len(df) == 0:
            print('No data found for {} from {} until {}'.format(ticker,start.strftime('%Y-%m-%d'),stop.strftime('%Y-%m-%d')),end="")
            failed_to_add.append(ticker)
            continue
        df1 = pd.DataFrame(df)
        df1['date'] = pd.to_datetime(df1['date'])
        df1.set_index('date',inplace=True)
        # merge the dataset
        price_df.loc[df1.index, ticker] = df1['adjusted_close']    

    #occasionally backup the dataframe
    if count % 20 == 0:
        price_df.to_csv(filename)

price_df.to_csv(filename) #save when done

  1%|          | 6/1156 [00:24<51:10,  2.67s/it]  

No data found for PZE from 1996-01-02 until 1998-12-31

  1%|          | 14/1156 [00:42<43:06,  2.27s/it]

No data found for DOW from 1996-01-02 until 2017-09-01

  2%|▏         | 23/1156 [01:11<1:06:07,  3.50s/it]

...

 99%|█████████▊| 1140/1156 [1:10:44<00:45,  2.83s/it]

No data found for MOB from 1996-01-02 until 1999-12-01

 99%|█████████▉| 1143/1156 [1:11:10<01:05,  5.02s/it]

No data found for COC.B from 1999-08-09 until 2002-09-03

100%|██████████| 1156/1156 [1:11:40<00:00,  3.72s/it]


### 2) dataset generated using fmp

In [8]:
import pickle
import os 
with open("fmp_key.pkl", "rb") as input_file:
  fmp_key = pickle.load(input_file)

with open("eod_key.pkl", "rb") as input_file:
  eod_key = pickle.load(input_file)

import urllib, json
import requests
from urllib.request import urlopen
import pandas as pd

# Function to get the Data

def get_jsonparsed_data(url):
    res = urlopen(url)
    data = res.read().decode("utf-8")
    return json.loads(data)

import datetime as dt
from tqdm import tqdm 

filename = 'price_df.csv'
data_provider  = 'eod' # other choice is 'fmp'

# first look up the historical s&p 500 constituents
constituents = get_jsonparsed_data("https://financialmodelingprep.com/api/v3/historical/sp500_constituent?apikey={}".format(fmp_key))
constituents = pd.DataFrame(constituents)
constituents['date'] = pd.to_datetime(constituents['date'])
all_tickers = set(constituents['symbol'].values)

begin = dt.datetime(1980,1,2)
end   = dt.datetime(2024,1,1)

price_df = pd.DataFrame(index=pd.date_range(start=begin, end=end, freq="B"))
failed_to_add = list()
# check if the file already exists
if os.path.exists(filename):
    price_df = pd.read_csv(filename)
    price_df = price_df.rename(columns={'Unnamed: 0':'date'})
    price_df.set_index('date', inplace=True)    

for count, ticker in enumerate(tqdm(all_tickers)):
    # find the correct dates the stock was part of the s&p 500
    date_begin = begin
    date_end   = end
    ticker = ticker.split(" ")[0]
    
    if ticker in price_df.columns:
        continue
    for _, row in constituents[constituents.symbol == ticker].iterrows():
        if len(row['addedSecurity']) != 0:
            date_begin = begin if row.date <= begin else row.date
        if len(row['removedSecurity']) != 0:
            date_end = end if row.date >= end else end

    if data_provider == 'fmp':
        # get the data from the fmp api
        df = get_jsonparsed_data("https://financialmodelingprep.com/api/v3/historical-price-full/{}?from={}&to={}&apikey={}".format(ticker,
            date_begin.strftime('%Y-%m-%d'),date_end.strftime('%Y-%m-%d'),fmp_key))
        if len(df) == 0:
            print('could not add {}'.format(ticker),end="")
            failed_to_add.append(ticker)
            continue
        df1 = pd.DataFrame(df['historical'])
        df1['date'] = pd.to_datetime(df1['date'])
        df1.set_index('date',inplace=True)
        # merge the dataset
        price_df = pd.merge(price_df, df1['adjClose'], how='left', left_index=True, right_index=True)
        price_df = price_df.rename(columns={'adjClose': ticker})
    elif data_provider == 'eod':
        df  = get_jsonparsed_data("https://eodhd.com/api/eod/{}.US?from={}&to={}&period=d&api_token={}&fmt=json".format(ticker,
            date_begin.strftime('%Y-%m-%d'),date_end.strftime('%Y-%m-%d'), eod_key))
        if len(df) == 0:
            print('could not add {}'.format(ticker),end="")
            failed_to_add.append(ticker)
            continue
        df1 = pd.DataFrame(df)
        df1['date'] = pd.to_datetime(df1['date'])
        df1.set_index('date',inplace=True)
        # merge the dataset
        price_df = pd.merge(price_df, df1['adjusted_close'], how='left', left_index=True, right_index=True)
        price_df = price_df.rename(columns={'adjusted_close': ticker})

    # occasionally backup the dataframe
    if count % 20 == 0:
        price_df.to_csv(filename)

KeyboardInterrupt: 